In [14]:
import polars

df = polars.read_csv("./data/Trade_DetailedTradeMatrix.csv", encoding='latin-1')

This is the part where we analyze the data.

In [61]:
import duckdb

duckdb.sql('''
    select distinct \"Reporter Countries\",
        count (\"Reporter Countries\") as n
        from \'./data/cleandata.csv\'
        where Year = 2015 and
        Element = 'Import Quantity' and
        Unit = 'tonnes'
        group by \"Reporter Countries\"
        order by n desc;
''')

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌──────────────────────────────────────────────────────┬───────┐
│                  Reporter Countries                  │   n   │
│                       varchar                        │ int64 │
├──────────────────────────────────────────────────────┼───────┤
│ Germany                                              │ 11444 │
│ Canada                                               │ 10092 │
│ France                                               │  9893 │
│ Netherlands                                          │  9513 │
│ Switzerland                                          │  9373 │
│ United Kingdom of Great Britain and Northern Ireland │  9046 │
│ United Arab Emirates                                 │  8657 │
│ Italy                                                │  8223 │
│ United States of America                             │  8220 │
│ Spain                                                │  7670 │
│   ·                                                  │    ·  │
│   ·                    